<a href="https://colab.research.google.com/github/alexsanderthorne/smartSystems/blob/main/Algoritmos_Gen%C3%A9ticos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from math import inf
import random
import numpy as np

class Individuo:

    def __init__(self, cidades, matriz_distancias):
        self.cromossomo = np.array(random.sample(range(cidades), cidades), dtype=int)
        self.matriz_distancias = matriz_distancias
        self.aptidao = self.fitness()

    def atualizar_cromossomo(self, novo_cromossomo):
        self.cromossomo = np.array(novo_cromossomo, dtype=int)
        self.aptidao = self.fitness()

    def fitness(self):
        custo = 0
        for i in range(len(self.cromossomo) - 1):
            cidade_atual = self.cromossomo[i]
            proxima_cidade = self.cromossomo[i + 1]
            custo += self.matriz_distancias[cidade_atual][proxima_cidade]
        custo += self.matriz_distancias[self.cromossomo[-1]][self.cromossomo[0]]  # Fechando o ciclo
        return custo

In [11]:
def ler_matriz_distancias(arquivo_tsp):
    with open(arquivo_tsp, 'r') as file:
        lines = file.readlines()
    
    # Modificando a leitura da matriz
    matriz_distancias = [list(map(int, line.split()[1:])) for line in lines[6:]]
    return np.array(matriz_distancias, dtype=int)

def iniciar_populacao(cidades, tamanho_populacao, matriz_distancias):
    return [Individuo(cidades, matriz_distancias) for _ in range(tamanho_populacao)]

def crossover_ox(pai1, pai2):
    ponto1 = random.randint(0, len(pai1.cromossomo) - 1)
    ponto2 = random.randint(ponto1 + 1, len(pai1.cromossomo))

    filho1 = [-1] * len(pai1.cromossomo)
    filho2 = [-1] * len(pai1.cromossomo)

    # Mantém a seção entre os pontos de corte dos pais nos filhos
    filho1[ponto1:ponto2] = pai1.cromossomo[ponto1:ponto2]
    filho2[ponto1:ponto2] = pai2.cromossomo[ponto1:ponto2]

    # Preenche o restante dos filhos com os genes do outro pai
    i = ponto2
    j = ponto2
    while -1 in filho1:
        if pai2.cromossomo[j % len(pai2.cromossomo)] not in filho1:
            filho1[i % len(pai1.cromossomo)] = pai2.cromossomo[j % len(pai2.cromossomo)]
            i += 1
        j += 1

    i = ponto2
    j = ponto2
    while -1 in filho2:
        if pai1.cromossomo[j % len(pai1.cromossomo)] not in filho2:
            filho2[i % len(pai1.cromossomo)] = pai1.cromossomo[j % len(pai1.cromossomo)]
            i += 1
        j += 1

    filho1_cromossomo = np.array(filho1)
    filho2_cromossomo = np.array(filho2)

    filho1 = Individuo(cidades, matriz_distancias)
    filho2 = Individuo(cidades, matriz_distancias)

    filho1.atualizar_cromossomo(filho1_cromossomo)
    filho2.atualizar_cromossomo(filho2_cromossomo)

    return filho1, filho2

def mutacao_swap(individuo):
    gene1 = random.randint(0, len(individuo.cromossomo) - 1)
    gene2 = random.randint(0, len(individuo.cromossomo) - 1)
    individuo.cromossomo[gene1], individuo.cromossomo[gene2] = individuo.cromossomo[gene2], individuo.cromossomo[gene1]
    individuo.cromossomo = np.array(individuo.cromossomo)  # Adicione esta linha


def melhor_individuo(populacao):
    return min(populacao, key=lambda x: x.aptidao)

In [ ]:
def algoritmo_genetico(n_geracoes, tamanho_populacao, cidades, matriz_distancias, taxa_mutacao):
    populacao = iniciar_populacao(cidades, tamanho_populacao, matriz_distancias)
    melhor_de_todos = melhor_individuo(populacao)
    solucoes_geracao = [melhor_de_todos.aptidao]

    for contador_geracoes in range(n_geracoes):
        nova_populacao = []

        for n_pop in range(0, tamanho_populacao, 2):
            indice_pai1 = selecao_torneio(populacao)
            indice_pai2 = selecao_torneio(populacao)
            filho1, filho2 = crossover_ox(populacao[indice_pai1], populacao[indice_pai2])

            mutacao_swap(filho1) if random.random() < taxa_mutacao else None
            mutacao_swap(filho2) if random.random() < taxa_mutacao else None

            nova_populacao.append(Individuo(cidades, matriz_distancias, filho1))
            nova_populacao.append(Individuo(cidades, matriz_distancias, filho2))

        populacao = nova_populacao
        melhor_ind_geracao = melhor_individuo(populacao)

        if melhor_de_todos.aptidao > melhor_ind_geracao.aptidao:
            melhor_de_todos = melhor_ind_geracao

        solucoes_geracao.append(melhor_de_todos.aptidao)
        print(f'[{contador_geracoes}]: {melhor_ind_geracao.cromossomo} = {melhor_ind_geracao.aptidao}')

    return melhor_de_todos, solucoes_geracao

def selecao_torneio(populacao):
    torneio = random.sample(range(len(populacao)), 3)
    return min(torneio, key=lambda x: populacao[x].aptidao)

# Configurações
arquivo_tsp = 'brazil58.tsp'
cidades = 58
tamanho_populacao = 100
n_geracoes = 100
taxa_mutacao = 0.1

# Execução do algoritmo genético
matriz_distancias = ler_matriz_distancias(arquivo_tsp)
melhor_sol, solucoes_gerais = algoritmo_genetico(n_geracoes, tamanho_populacao, cidades, matriz_distancias, taxa_mutacao)

# Imprimir a melhor solução encontrada
print(f'\nMelhor solução: {melhor_sol.cromossomo} = {melhor_sol.aptidao}')